#### Load Libraries

In [ ]:
import pandas as pd
import numpy as np


### Load Data

### Exploratory Data Analysis

You can leverage what done in one of the previous assignments, recreate it and expand it.

### Feature Engineering and Analysis
Think about any possible feature that is predictive of the target.  
Measure how each feature is predictive of the target and select all those relevant for the model.

### Configure the model using an appropriate classification technique

### Chose the correct metric for assessment of the model